<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

# Performance

We have not really paid much attention to performance of code beyond basic instruction counting up until now.  These lectures are dedicated to a deeper dive into these issues and what are some important things to keep in mind.

## Example: Matrix-Matrix Multiplication

To start our discussion let us consider the algorithm for matrix-matrix multiplication which algorithmically looks like
```
do i=1:N
    do j=1:N
        do k=1:N
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end do
    end do
end do
```

Consider the follow approaches to this problem:
1. Matrix multiplication via a GCC (Fortran) intrinsic.
1. Straight forward three-loop multiplication
1. Parallelized double-loop using BLAS intrinsic
1. BLAS intrinsic

```fortran
real function matrix_multiply_test(N,method)

    use mod_rand
    implicit none
    
    external DGEMM,DDOT
    
    double precision :: DDOT
    integer, intent(in) :: N,method
    integer :: start,finish,count_rate
    double precision, dimension(:,:), allocatable :: A,B,C
    
    ! Local
    integer :: i,j,k
    
    ! Create the random arrays
    call init_random_seed()
    allocate(A(N,N),B(N,N),C(N,N))
    call random_number(A)
    call random_number(B)
    
    ! Start the timer and start multiplying
    call system_clock(start,count_rate)
    select case(method)
        case(1) ! Default method provided as an intrinsic method
            C = matmul(A,B)
        case(2) ! Simple three loop multiplication
            !$OMP parallel do private(j,k)
            do i=1,N
                do j=1,N
                    do k=1,N
                        C(i,j) = C(i,j) + A(i,k) * B(k,j)
                    enddo
                enddo
            enddo
        case(3) ! OpenMP parallelized double loop
            !$OMP parallel do private(j)
            do i=1,N
                do j=1,N
                    C(i,j) = DDOT(N, A(i,:), 1, B(:,j), 1)
                enddo
            enddo
        case(4) ! BLAS Routine call
            ! call DGEMM(transa,transb,l,n,m,alpha,a,lda,b,ldb,beta,c,ldc)
            call DGEMM('N', 'N', N, N, N, 1.d0, A, N, B, N, 0.d0, C, N)
        case default
            print *, "***ERROR*** Invalid multiplication method!"
            matrix_multiply_test = -1
            return
    end select
    call system_clock(finish,count_rate)
    
    matrix_multiply_test = float(finish - start) / float(count_rate)
    
end function matrix_multiply_test
    
program matrix_multiply
    
    use omp_lib

    implicit none
    
    integer :: N, method, threads
    character(len=10) :: input_N, input_method, input_threads
    real :: matrix_multiply_test, time
    
    select case(iargc())
        case(0)
            N = 1000
            method = 1
            threads = 1
        case(1)
            call getarg(1,input_N)
            read(input_N,'(I10)') N
            method = 1
        case(2)
            call getarg(1,input_N)
            call getarg(2,input_method)
            read(input_N,'(I10)') N
            read(input_method,'(I10)') method
        case(3)
            call getarg(1,input_N)
            call getarg(2,input_method)
            call getarg(3,input_threads)
            read(input_N,'(I10)') N
            read(input_method,'(I10)') method
            read(input_threads,'(I10)') threads
        case default
            print *, "***ERROR*** Too many arguments!"
            stop
    end select
    
    !$ call omp_set_num_threads(threads)

    time = matrix_multiply_test(N, method)
    
    print '("Timing for ",i5,"x",i5," matrices: ",f10.5," s")',N,N,time
    
end program matrix_multiply
```

#### Results

Based on $1000 \times 1000$ matrix-matrix multiply compiled with `gfortran` version 6.3.0 with the compile time flags `-O3 -funroll-loops -finline-functions -fdefault-real-8 -fopenmp`.


Method                           | No-Threads            | Threaded
---------------------------------|-----------------------|---------------------------
Default mat_mult function        |            35.79600 s |                36.19100 s
3 loop multiplication            |            39.24700 s |                10.04000 s                   
Double loop (internal BLAS)      |             6.80500 s |                 1.76600 s                   
BLAS Routine call                |             0.00300 s |                 0.00300 s                   


## Machine Architecture Considerations

Before we can talk about performance we need to understand a bit about modern computing architectures.  Note that this glossing over a lot of important details as we will focus on only the details that we will specifically deal with.

### Von Neumann Architecture
![image](./images/vonneumann_architecture.png)
*Kapooht - Wikipedia Commons*

### Instruction Pipeline
![image](./images/pipeline_1.png)
*Cburnett - Wikipedia Commons*

![image](./images/memory_architecture.png)

### Moore's Law

In 1965, Gordon Moore (co-founder of Intel) predicted that the transistor density (and hence the speed) of chips would double every 18 months for the forseeable future. This is know as Moore’s law This proved remarkably accurate for more than 40 years, see the graphs at. Note that doubling every 18 months means an increase by a factor of 4096 every 14 years.

### Is Moore's Law Doomed?
![image](./images/moores_law.png)
*Steve Jurvetson - Wikipedia Commons - https://www.flickr.com/photos/jurvetson/31409423572/*

### Current Performance Bottlenecks

 - Transistors can no longer be packed more densely in a single core
 - Memory is really the bottleneck
 - Hard limit due to the speed of light
 - Power consumption and therefore heat dissipation
 
### Solutions?
 - Many-core technologies
 - Memory hierarchies
 - Algorithms that take into account these limitations

![image](./images/memory_single_core.png)

![image](./images/pipeline_2.png)
*Cburnett - Wikipedia Commons*

### Roof-line Model
![image](./images/roofline.png)
*Giu.natale - Wikipedia Commons*

### Many-Core Architectures

![image](./images/kepler_arch.png)
![image](./images/kepler_smx.png)
*NVIDIA - Kepler GK110/210 White Paper - http://images.nvidia.com/content/pdf/tesla/NVIDIA-Kepler-GK110-GK210-Architecture-Whitepaper.pdf*

## Parallelization

Parallelization is one of the primary ways we can increase performance on today's computing architectures.  There are 2+1 major types of parallelization paradigms:
 - Shared memory - each pipeline can access the memory for the entire problem
 - Distributed memory - each pipeline can only access part of the memory for the entire problem
 - Hybrid parallelism - use both...

### Shared Memory

 - Basic construct is a *thread* - each thread has a pipeline and in the simplest case each core runs one thread
 - OpenMP, CUDA, OpenCL, OpenACC
 - Single nodes on a cluster, GPU, Xeon Phi, etc.

### Distributed Memory

 - Basic contruct is a *process* 
 - Each process is memory local but can communicate to other processes either on the same CPU or across a network
 - Each process can have multiple threads (hybrid parallelism)
 - MPI is most common
 - Clusters, super-computers, etc.

### Scalability

Measures of parallel performance:

 - Strong Scaling:  Execution time decreases inversely proportional to the number of processes
   - Fixed size problem
 - Weak Scaling: Execution time remains constant as problem size and processes number are increased proportionally
   - Variable size problem

### OpenMP

OpenMP is defined by a set of *directives* that are put into code which on compilation a compiler can turn into multi-threaded code.

```fortran
program yeval
   
   use omp_lib

   implicit none

   integer(kind=8), parameter :: n = 2**16
   integer(kind=4) :: i, nthreads
   real(kind=8), dimension(n) :: y
   real(kind=8) :: dx, x

   ! Specify number of threads to use:
   !$ print *, "How many threads to use? "
   !$ read *, nthreads
   !$ call omp_set_num_threads(nthreads)
   !$ print "('Using OpenMP with ',i3,' threads')", nthreads

   dx = 1.d0 / (n+1.d0)

   !$omp parallel do private(x) 
   do i=1, n
      x = i * dx
      y(i) = exp(x) * cos(x) * sin(x) * sqrt(5.d0 * x + 6.d0)
   enddo
   !$omp end parallel do

   print *, "Filled vector y of length", n

end program yeval
```
*Modified from amath 583 - R.J. LeVeque - http://faculty.washington.edu/rjl/classes/am583s2014/notes/openmp.html*

#### Fine-Grain vs. Coarse-Grain Parallelism

Consider the problem of normalizing a vector which requires two steps:
1. Compute the norm of the vector, and
1. Divide each entry of the vector by the norm.

Unfortunately we need to loop over every entry in the vector to compute the norm **before** we can perform the division of each entry.  There are two ways to tackle this problem,
 - Let the compiler decide what thread takes what entries (fine grain)
 - Let the programmer explicitly control which entries are handled by each thread (coarse grain)

```fortran
program fine_grain
   
    use omp_lib
    implicit none
    integer :: i, thread_num
    integer, parameter :: n = 1000
 
    real(kind=8), dimension(n) :: x, y
    real(kind=8) :: norm,ynorm
 
    integer :: nthreads
    
    ! Specify number of threads to use:
    nthreads = 1       ! need this value in serial mode
    !$ nthreads = 4    
    !$ call omp_set_num_threads(nthreads)
    !$ print "('Using OpenMP with ',i3,' threads')", nthreads

    ! Specify number of threads to use:
    !$ call omp_set_num_threads(4)
 
    ! initialize x:
    !$omp parallel do 
    do i=1,n
        x(i) = real(i, kind=8)  ! convert to double float
    enddo

    norm = 0.d0
    ynorm = 0.d0

    !$omp parallel private(i)

    !$omp do reduction(+ : norm)
    do i=1,n
        norm = norm + abs(x(i))
    enddo

     !$omp barrier   ! not needed (implicit)

    !$omp do reduction(+ : ynorm)
    do i=1,n
        y(i) = x(i) / norm
        ynorm = ynorm + abs(y(i))
    enddo
    
    !$omp end parallel

    print *, "norm of x = ",norm, "  n(n+1)/2 = ",n*(n+1)/2
    print *, 'ynorm should be 1.0:   ynorm = ', ynorm

end program fine_grain

```
*Modified from amath 583 - R.J. LeVeque - http://faculty.washington.edu/rjl/classes/am583s2014/notes/openmp.html*

```fortran
program coarse_grain
    
    use omp_lib
    implicit none
    integer, parameter :: n = 1000
    real(kind=8), dimension(n) :: x,y
    real(kind=8) :: norm,norm_thread,ynorm,ynorm_thread
    integer :: nthreads, points_per_thread,thread_num
    integer :: i,istart,iend

    ! Specify number of threads to use:
    nthreads = 1       ! need this value in serial mode
    !$ nthreads = 4    
    !$ call omp_set_num_threads(nthreads)
    !$ print "('Using OpenMP with ',i3,' threads')", nthreads

    ! Determine how many points to handle with each thread.
    ! Note that dividing two integers and assigning to an integer will
    ! round down if the result is not an integer.  
    ! This, together with the min(...) in the definition of iend below,
    ! insures that all points will get distributed to some thread.
    points_per_thread = (n + nthreads - 1) / nthreads
    print *, "points_per_thread = ",points_per_thread

    ! initialize x:
    do i=1,n
        x(i) = dble(i)  ! convert to double float
        enddo

    norm = 0.d0
    ynorm = 0.d0

    !$omp parallel private(i,norm_thread, &
    !$omp                  istart,iend,thread_num,ynorm_thread) 

    thread_num = 0     ! needed in serial mode
    !$ thread_num = omp_get_thread_num()    ! unique for each thread

    ! Determine start and end index for the set of points to be 
    ! handled by this thread:
    istart = thread_num * points_per_thread + 1
    iend = min((thread_num+1) * points_per_thread, n)

    !$omp critical
    print "("Thread ",i2," will take i = ",i6," through i = ",i6)", thread_num, istart, iend
    !$omp end critical

    norm_thread = 0.d0
    do i=istart,iend
        norm_thread = norm_thread + abs(x(i))
        enddo

    ! update global norm with value from each thread:
    !$omp critical
      norm = norm + norm_thread
      print *, "norm updated to: ",norm
    !$omp end critical

    ! make sure all have updated norm before proceeding:
    !$omp barrier

    ynorm_thread = 0.d0
    do i=istart,iend
        y(i) = x(i) / norm
        ynorm_thread = ynorm_thread + abs(y(i))
        enddo

    ! update global ynorm with value from each thread:
    !$omp critical
      ynorm = ynorm + ynorm_thread
      print *, "ynorm updated to: ",ynorm
    !$omp end critical
    !$omp barrier

    !$omp end parallel 

    print *, "norm of x = ",norm, "  n(n+1)/2 = ",n*(n+1)/2
    print *, 'ynorm should be 1.0:   ynorm = ', ynorm

end program coarse_grain
```
*Modified from amath 583 - R.J. LeVeque - http://faculty.washington.edu/rjl/classes/am583s2014/notes/openmp.html*

### MPI

*Modified from amath 583 - R.J. LeVeque - http://faculty.washington.edu/rjl/classes/am583s2014/notes/mpi.html*

### Example - Jacobi Iteration

http://faculty.washington.edu/rjl/classes/am583s2014/notes/jacobi1d_omp1.html
http://faculty.washington.edu/rjl/classes/am583s2014/notes/jacobi1d_omp2.html
http://faculty.washington.edu/rjl/classes/am583s2014/notes/jacobi1d_mpi.html

## Software Packages

### Linear Algebra
 - BLAS - Basic Lineag Algebra Subroutines
   - Level 1: Scalar and vector operations
   - Level 2: Matrix-vector operations
   - Level 3: Matrix-matrix operations
 - LAPACK - Linear Algebra Package
 - ScaLAPACK - Parallel LAPACK

### Solving ODEs
ODEPACK

### Solving PDEs